In [1]:
#/content/drive/MyDrive/OmdenaHOTOSM_202407/convert_to_3857/convert_to_3857

In [2]:
!pip install rasterio -q #quiet install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 21.3 MB/s eta 0:00:00


In [3]:
import numpy as np

import rasterio
from rasterio import Affine as A
from rasterio.warp import reproject, Resampling
from rasterio.warp import calculate_default_transform, reproject, Resampling


In [4]:
def reproject_raster(source_img,
                     output_path,
                     source_crs,
                     target_crs):

  dst_crs = 'EPSG:3857'

  transform, width, height = calculate_default_transform(
      source_crs, target_crs, source_img.width, source_img.height, *source_img.bounds)
  print(transform)
  print(width)
  print(height)

  kwargs = source_img.meta.copy()
  kwargs.update({
      'crs': target_crs,
      'transform': transform,
      'width': width,
      'height': height
  })

  with rasterio.open(output_path, 'w', **kwargs) as dst:
      for i in range(1, source_img.count + 1):
          reproject(
              source=rasterio.band(source_img, i),
              destination=rasterio.band(dst, i),
              src_transform=source_img.transform,
              src_crs=source_crs,
              dst_transform=transform,
              dst_crs=dst_crs,
              resampling=Resampling.nearest)

In [17]:
def reproject_raster_fixed_size(source_img, output_path, source_crs, target_crs):
  # Calculate new pixel resolution based on original dimensions and desired CRS
  left, bottom, right, top = source_img.bounds
  projected_bounds = rasterio.warp.transform_bounds(source_crs, target_crs, left, bottom, right, top)

  # Calculate new pixel resolution based on projected bounds and original dimensions
  target_transform = rasterio.transform.from_bounds(*projected_bounds, source_img.width, source_img.height)
  print(target_transform)
  print(source_img.width)
  print(source_img.height)

  kwargs = source_img.meta.copy()
  kwargs.update({
      'crs': target_crs,
      'transform': target_transform,  # Use the calculated transform
  })

  with rasterio.open(output_path, 'w', **kwargs) as dst:
      for i in range(1, source_img.count + 1):
          reproject(
              source=rasterio.band(source_img, i),
              destination=rasterio.band(dst, i),
              src_transform=source_img.transform,
              src_crs=source_crs,
              dst_transform=target_transform,  # Use the calculated transform
              dst_crs=target_crs,
              resampling=Resampling.nearest)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
directories = []

#correct this path to where the images are stored:
folder_path = '/content/drive/MyDrive/OmdenaHOTOSM_202407/convert_to_3857/convert_to_3857'

items = os.listdir(folder_path)

for item in items:
    item_path = os.path.join(folder_path, item)
    if os.path.isdir(item_path):
        #print(f"'{item}' is a directory.")
        directories.append(item)
    elif os.path.isfile(item_path):
        print(f"'{item}' is a file.")
print(directories)

folder_path2 = os.path.join(folder_path, '3857')
folder_path2b = os.path.join(folder_path, '3857_v2')

new_folders = [folder_path2, folder_path2b]

# create the output directory, in not exists
for folder in new_folders:
  if not os.path.exists(folder):
    os.makedirs(folder)
    print(f"Folder '{folder}' created successfully.")
  else:
    print(f"Folder '{folder}' already exists.")


'.DS_Store' is a file.
['4326', '32646', '32737', '32630', '3857']
Folder '/content/drive/MyDrive/OmdenaHOTOSM_202407/convert_to_3857/convert_to_3857/3857' already exists.
Folder '/content/drive/MyDrive/OmdenaHOTOSM_202407/convert_to_3857/convert_to_3857/3857_v2' created successfully.


In [10]:

# Simple reprojection, size will be altered to find the best matching dx/dy
for directory in directories:
  print(directory)
  if directory not in ['3857', '3857_v2']:
    folder_path3 = os.path.join(folder_path, directory)
    #print(folder_path3)
    items = os.listdir(folder_path3)
    for item in items:
      item_path = os.path.join(folder_path3, item)
      #print(item_path)
      if os.path.isfile(item_path):
        print(f"'{item}' is a file.")
        _, extension = os.path.splitext(item)
        if extension == '.tif':
            print("The file is a TIF file.")
            # now get the CRS

            with rasterio.open(item_path) as src:
                src_crs = src.crs
                print(src_crs)

                dst_crs = 'EPSG:3857'
                #dst_crs = 'EPSG:4326'
                item_path2 = os.path.join(folder_path2, item)

                reproject_raster(src, item_path2, src_crs, dst_crs)

        else:
            print("The file is not a TIF file.")


4326
'0aaa8aa8-b099-402a-a4a0-6647fa79228c.tif' is a file.
The file is a TIF file.
EPSG:4326
| 0.46, 0.00, 10266824.87|
| 0.00,-0.46, 2396526.17|
| 0.00, 0.00, 1.00|
247
265
'0aea67bc-e36b-4e73-87a9-34307099a74e.tif' is a file.
The file is a TIF file.
EPSG:4326
| 0.44, 0.00, 10057617.01|
| 0.00,-0.44, 2592412.29|
| 0.00, 0.00, 1.00|
246
266
'0c913da4-40e4-4b17-9d8b-e72b971db5b9.tif' is a file.
The file is a TIF file.
EPSG:4326
| 0.36, 0.00, 3316130.78|
| 0.00,-0.36, 1033253.87|
| 0.00, 0.00, 1.00|
254
258
32646
'0a6de8c0-e4d8-4fae-ab55-21bfd7f62022.tif' is a file.
The file is a TIF file.
EPSG:32646
| 0.33, 0.00, 10063870.80|
| 0.00,-0.33, 2739683.58|
| 0.00, 0.00, 1.00|
260
261
'0a06e8fe-7573-4f89-bba9-08c6e864ed34.tif' is a file.
The file is a TIF file.
EPSG:32646
| 0.33, 0.00, 10067042.95|
| 0.00,-0.33, 2717307.54|
| 0.00, 0.00, 1.00|
260
261
'0a6df8ab-a04b-46ba-bc06-9677cb74cafc.tif' is a file.
The file is a TIF file.
EPSG:32646
| 0.33, 0.00, 10064723.57|
| 0.00,-0.33, 2738939.73|
|

In [18]:

# Fixing the size to the source, this will result in the dx/dy to be different
for directory in directories:
  print(directory)
  if directory not in ['3857', '3857_v2']:
    folder_path3 = os.path.join(folder_path, directory)
    #print(folder_path3)
    items = os.listdir(folder_path3)
    for item in items:
      item_path = os.path.join(folder_path3, item)
      #print(item_path)
      if os.path.isfile(item_path):
        print(f"'{item}' is a file.")
        _, extension = os.path.splitext(item)
        if extension == '.tif':
            print("The file is a TIF file.")
            # now get the CRS

            with rasterio.open(item_path) as src:
                src_crs = src.crs
                print(src_crs)

                dst_crs = 'EPSG:3857'
                #dst_crs = 'EPSG:4326'
                item_path2b = os.path.join(folder_path2b, item)

                #reproject_raster(src, item_path2, src_crs, dst_crs)
                reproject_raster_fixed_size(src, item_path2b, src_crs, dst_crs)

        else:
            print("The file is not a TIF file.")


4326
'0aaa8aa8-b099-402a-a4a0-6647fa79228c.tif' is a file.
The file is a TIF file.
EPSG:4326
| 0.45, 0.00, 10266824.87|
| 0.00,-0.48, 2396526.17|
| 0.00, 0.00, 1.00|
256
256
'0aea67bc-e36b-4e73-87a9-34307099a74e.tif' is a file.
The file is a TIF file.
EPSG:4326
| 0.43, 0.00, 10057617.01|
| 0.00,-0.46, 2592412.29|
| 0.00, 0.00, 1.00|
256
256
'0c913da4-40e4-4b17-9d8b-e72b971db5b9.tif' is a file.
The file is a TIF file.
EPSG:4326
| 0.35, 0.00, 3316130.78|
| 0.00,-0.36, 1033253.87|
| 0.00, 0.00, 1.00|
256
256
32646
'0a6de8c0-e4d8-4fae-ab55-21bfd7f62022.tif' is a file.
The file is a TIF file.
EPSG:32646
| 0.33, 0.00, 10063870.80|
| 0.00,-0.34, 2739683.58|
| 0.00, 0.00, 1.00|
256
256
'0a06e8fe-7573-4f89-bba9-08c6e864ed34.tif' is a file.
The file is a TIF file.
EPSG:32646
| 0.33, 0.00, 10067042.95|
| 0.00,-0.34, 2717307.54|
| 0.00, 0.00, 1.00|
256
256
'0a6df8ab-a04b-46ba-bc06-9677cb74cafc.tif' is a file.
The file is a TIF file.
EPSG:32646
| 0.33, 0.00, 10064723.57|
| 0.00,-0.34, 2738939.73|
|